In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from tqdm import tqdm
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Point
from pyproj import Transformer
import ipyparallel as ipp
from scipy.stats import linregress
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.linear_model import LinearRegression
import xml.etree.ElementTree as ET
import json
from pyproj import Transformer
from sklearn.decomposition import PCA
import pickle

In [2]:
s3_sar_5021 = "https://ds-h-ca-bigdata.s3.us-east-2.amazonaws.com/CA_DWR_VERT.csv"
sar_5021 = pd.read_csv(s3_sar_5021)

In [3]:
with open("grid_large_std.pkl", "rb") as f:
    grid = pickle.load(f)

In [4]:
dry = ["2017", "2019", "2020", "2021"]
wet = ["2015", "2016", "2018", "2022", "2023"]

In [5]:
def is_oct_2015_or_later(col_name):
    if col_name.startswith('D') and len(col_name) == 9:
        try:
            date_str = col_name[1:]
            date = datetime.strptime(date_str, '%Y%m%d')
            return date >= datetime(2015, 10, 1)
        except ValueError:
            return False
    return True

In [6]:
wy_2015_or_later = [col for col in sar_5021.columns if is_oct_2015_or_later(col)]

In [7]:
sar_5021_2015 = sar_5021[wy_2015_or_later].copy()

In [8]:
def extract_year_month(col_name):
    if col_name.startswith('D') and len(col_name) == 9:
        try:
            date_str = col_name[1:]
            date = datetime.strptime(date_str, '%Y%m%d')
            return date.year, date.month
        except ValueError:
            pass
    return None, None

year_month_columns = {}
for col in sar_5021_2015.columns:
    year, month = extract_year_month(col)
    if year is not None:
        year_month_columns.setdefault((year, month), []).append(col)

def calculate_average(row, columns):
    valid_values = [row[col] for col in columns if not pd.isna(row[col])]
    return np.mean(valid_values) if valid_values else np.nan

results = []
for index, row in tqdm(sar_5021_2015.iterrows(), total=len(sar_5021_2015), desc="Processing rows"):
    row_results = {'CODE': row['CODE']}
    for (year, month), columns in year_month_columns.items():
        if month == 10:
            oct_avg = calculate_average(row, columns)
            next_year_sep_columns = year_month_columns.get((year + 1, 9), [])
            sep_avg = calculate_average(row, next_year_sep_columns)
            if not np.isnan(oct_avg) and not np.isnan(sep_avg):
                row_results[f'Diff_{year}'] = oct_avg - sep_avg
    results.append(row_results)

wy_sar_5021 = pd.DataFrame(results)

Processing rows: 100%|██████████| 707344/707344 [04:56<00:00, 2389.25it/s]


In [9]:
def extract_year(col_name):
    if col_name.startswith('D') and len(col_name) == 9:
        return col_name.split("_")[1]
    return None

dry_columns = [col for col in wy_sar_5021.columns if extract_year(col) in dry]
wet_columns = [col for col in wy_sar_5021.columns if extract_year(col) in wet]

print("Dry Year Columns:", dry_columns)
print("Wet Year Columns:", wet_columns)

results = []
for idx, row in tqdm(wy_sar_5021.iterrows(), total=len(wy_sar_5021), desc="Processing rows"):
    well_code = row['CODE']
    dry_avg = row[dry_columns].mean()
    wet_avg = row[wet_columns].mean()
    results.append({
        'CODE': well_code,
        'Dry_Year_Avg': dry_avg,
        'Wet_Year_Avg': wet_avg
    })

wy_sar_5021_dry_wet = pd.DataFrame(results)

Dry Year Columns: ['Diff_2017', 'Diff_2020', 'Diff_2021', 'Diff_2019']
Wet Year Columns: ['Diff_2015', 'Diff_2016', 'Diff_2018', 'Diff_2022', 'Diff_2023']


Processing rows: 100%|██████████| 707344/707344 [05:32<00:00, 2127.88it/s]


In [10]:
wy_sar_5021_dry_wet = pd.merge(wy_sar_5021_dry_wet, sar_5021[["CODE", "LAT", "LON"]], on="CODE", how='left')

In [11]:
wy_sar_5021_dry_wet['geometry'] = wy_sar_5021_dry_wet.apply(lambda row: Point(row.LON, row.LAT), axis=1)

gdf = gpd.GeoDataFrame(wy_sar_5021_dry_wet, geometry='geometry')
gdf.crs = "EPSG:4326"

gdf = gdf.to_crs(epsg=32612)

In [12]:
grid = grid.to_crs(gdf.crs)
grid['geometry_buffer'] = grid.geometry.buffer(1000)

In [13]:
grid_buffered = grid.set_geometry('geometry_buffer')
joined = gpd.sjoin(gdf, grid_buffered, predicate='within')

In [14]:
averages = joined.groupby('index_right')[['Dry_Year_Avg', 'Wet_Year_Avg']].mean().reset_index()
grid_avg = grid.merge(averages, left_index=True, right_on='index_right', how='left')

In [15]:
mean_dry = grid_avg["Dry_Year_Avg"].mean()
std_dry = grid_avg["Dry_Year_Avg"].std()
grid_avg["Dry_Year_Avg_Z"] = (grid_avg["Dry_Year_Avg"] - mean_dry) / std_dry

mean_wet = grid_avg["Wet_Year_Avg"].mean()
std_wet = grid_avg["Wet_Year_Avg"].std()
grid_avg["Wet_Year_Avg_Z"] = (grid_avg["Wet_Year_Avg"] - mean_wet) / std_wet

In [16]:
grid_avg["Difference"] = grid_avg["Wet_Year_Avg"] - grid_avg["Dry_Year_Avg"]
mean_diff = grid_avg["Difference"].mean()
std_diff = grid_avg["Difference"].std()
grid_avg["Difference_Z"] = (grid_avg["Difference"] - mean_diff) / std_diff

In [17]:
grid_avg.drop(columns=["Dry_Year_Avg", "Wet_Year_Avg", "Difference"], inplace=True)

In [18]:
cvhm = pd.read_csv("GA-sklearn/cvhm_PC.csv")[["X_COORD", "Y_COORD", "PC_D25"]]

In [19]:
cvhm_gdf = gpd.GeoDataFrame(cvhm, geometry=gpd.points_from_xy(cvhm.X_COORD, cvhm.Y_COORD))

crs_albers = {
    'proj': 'aea',           # Albers Equal Area Conic projection
    'lat_1': 29.5,           # First standard parallel
    'lat_2': 45.5,           # Second standard parallel
    'lat_0': 23.0,           # Latitude of origin
    'lon_0': -120.0,         # Central meridian
    'x_0': 0,                # False easting
    'y_0': 0,                # False northing
    'datum': 'NAD83',        # Datum
    'units': 'm'             # Linear unit in meters
}

cvhm_gdf.set_crs(crs_albers, inplace=True)

,X_COORD,Y_COORD,PC_D25,geometry
0,-193663.0,1964850.0,25.29,POINT (-193663 1964850)
1,-195623.0,1961830.0,30.11,POINT (-195623 1961830)
2,-194235.0,1962650.0,26.48,POINT (-194235 1962650)
3,-192847.0,1963460.0,23.13,POINT (-192847 1963460)
4,-191460.0,1964280.0,16.90,POINT (-191460 1964280)
...,...,...,...,...
20528,107958.0,1324510.0,43.18,POINT (107958 1324510)
20529,109345.0,1325320.0,42.67,POINT (109345 1325320)
20530,110733.0,1326140.0,41.85,POINT (110733 1326140)
20531,104611.0,1320680.0,42.11,POINT (104611 1320680)


In [20]:
cvhm_gdf = cvhm_gdf.to_crs(grid_buffered.crs)
joined = gpd.sjoin(cvhm_gdf, grid_buffered, predicate='within')
avg_pc_d25 = joined.groupby('index_right')['PC_D25'].mean()
grid_buffered['avg_PC_D25'] = avg_pc_d25

In [21]:
grid_avg = grid_avg.join(grid_buffered[['avg_PC_D25']])

In [22]:
grid_avg['avg_PC_D25'] = grid_avg['avg_PC_D25'].fillna(-999)
valid_mask = grid_avg['avg_PC_D25'] != -999
valid_mean = grid_avg.loc[valid_mask, 'avg_PC_D25'].mean()
valid_std = grid_avg.loc[valid_mask, 'avg_PC_D25'].std()
grid_avg.loc[valid_mask, 'avg_PC_D25_z'] = (grid_avg.loc[valid_mask, 'avg_PC_D25'] - valid_mean) / valid_std
grid_avg.loc[~valid_mask, 'avg_PC_D25_z'] = -999

In [23]:
grid_avg.drop(columns=["avg_PC_D25"], inplace=True)

In [24]:
cols = ['Dry_Year_Avg_Z', 'Wet_Year_Avg_Z', 'Difference_Z', 'avg_PC_D25_z']
for col in cols:
    grid_avg[f'{col}_flag'] = (grid_avg[col] == -999).astype(int)

In [33]:
grid_avg.replace(-9999, 0, inplace=True)
grid_avg.replace(-999, 0, inplace=True)

In [2]:
with open('grid_large_std_v2.pkl', 'rb') as file:
    grid_avg = pickle.load(file)

In [3]:
NHD_CA = gpd.read_file("NHD_CA.gpkg")

In [4]:
NHD_CA[["fcode", "fcode_description"]].value_counts()

fcode  fcode_description                                                               
48400  Wash                                                                                8947
46006  Stream/River: Hydrographic Category = Perennial                                     1797
40300  Inundation Area                                                                     1622
36400  Foreshore                                                                           1427
33600  Canal/Ditch                                                                         1287
40308  Inundation Area: Inundation Control Status = Controlled                             1066
43100  Rapids                                                                               358
40309  Inundation Area: Inundation Control Status = Controlled; Stage = Flood Elevation     186
46003  Stream/River: Hydrographic Category = Intermittent                                   183
33601  Canal/Ditch: Canal/Ditch Type = Aqueduct 

In [5]:
NHD_CA = NHD_CA.to_crs(grid_avg.crs)

In [6]:
SGMA = "../data/GWBasins.shp"
gdf = gpd.read_file(SGMA)
gdf.set_crs(epsg=3857, inplace=True)
gdf.set_index('OBJECTID', inplace=True)
gdf['Basin_Prefix'] = gdf['Basin_Numb'].str.split('-').str[0].astype(int)
gdf = gdf[gdf['Basin_Numb'] == '5-021']

In [7]:
gdf = gdf.to_crs(NHD_CA.crs)

In [8]:
NHD_CA = NHD_CA[NHD_CA.geometry.intersects(gdf.unary_union)]

/tmp/ipykernel_60328/1251911098.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  NHD_CA = NHD_CA[NHD_CA.geometry.intersects(gdf.unary_union)]


In [9]:
water_5021 = NHD_CA.unary_union

/tmp/ipykernel_174662/3088505289.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  water_5021 = NHD_CA.unary_union


In [9]:
union_dict = {}
code_description_dict = {}

for (fcode, description), group in tqdm(NHD_CA.groupby(["fcode", "fcode_description"])):
    union_geom = group.geometry.unary_union
    union_dict[fcode] = union_geom
    code_description_dict[fcode] = description

  0%|          | 0/18 [00:00<?, ?it/s]/tmp/ipykernel_60328/2501136280.py:5: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  union_geom = group.geometry.unary_union
100%|██████████| 18/18 [00:02<00:00,  6.72it/s]


In [11]:
union_dict.keys()

dict_keys([33600, 33601, 34300, 34306, 36400, 39800, 40300, 40307, 40308, 44500, 46000, 46003, 46006, 46007, 46100, 48400, 53700, 56800])

In [ ]:
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib

def func1(fcode):
    shape = union_dict.get(fcode)
    mask = []
    for index, row in tqdm(grid_avg.iterrows()):
        if row.geometry.intersects(shape):
            mask.append(1)
        else:
            mask.append(0)
    return mask

fcodes = list(union_dict.keys())

results = Parallel(n_jobs=-1, verbose=10)(
    delayed(func1)(num) 
    for num in fcodes
)

with open('water_masks.pkl', 'wb') as file:
    pickle.dump(results, file)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
26413it [00:56, 468.00it/s] 
26413it [00:56, 466.19it/s] 
26413it [00:57, 459.32it/s] 
26413it [00:56, 465.13it/s] 
26413it [00:56, 465.09it/s] 
26413it [00:56, 468.31it/s] 
26413it [00:57, 461.26it/s] 
20836it [01:08, 287.00it/s][Parallel(n_jobs=-1)]: Done   3 out of  18 | elapsed:  1.3min remaining:  6.6min
26413it [00:57, 462.81it/s] 
26413it [00:56, 470.98it/s] 
21538it [01:09, 415.13it/s][Parallel(n_jobs=-1)]: Done   5 out of  18 | elapsed:  1.3min remaining:  3.5min
26413it [00:56, 469.65it/s] 
11303it [01:03, 33.28it/s][Parallel(n_jobs=-1)]: Done   7 out of  18 | elapsed:  1.4min remaining:  2.2min
1085it [01:15,  7.37it/s][Parallel(n_jobs=-1)]: Done   9 out of  18 | elapsed:  1.4min remaining:  1.4min
26413it [01:04, 409.95it/s] 
1163it [01:20, 10.32it/s][Parallel(n_jobs=-1)]: Done  11 out of  18 | elapsed:  1.5min remaining:   57.0s
26413it [01:20, 329.85it/s] 
26413it [01:53, 232.28it/s] 
1970it [02:

In [10]:
with open('water_masks.pkl', 'rb') as file:
    water_masks = pickle.load(file)

In [15]:
flags = dict(zip(list(union_dict.keys()), water_masks))

In [20]:
grid_avg = grid_avg.assign(**{str(key): value for key, value in flags.items()})

In [41]:
with open('grid_large_std_v2.pkl', 'wb') as file:
    pickle.dump(grid_avg, file)

In [40]:
cols_to_fill = ['Dry_Year_Avg_Z', 'Wet_Year_Avg_Z', 'Difference_Z']
grid_avg[cols_to_fill] = grid_avg[cols_to_fill].fillna(0)

In [3]:
grid_avg.memory_usage(deep=True).sum() / (1024 ** 2)

439.8572816848755

In [29]:
with open('grid_large_std_v2.pkl', 'rb') as file:
    grid_avg = pickle.load(file)

In [3]:
with open('grid_large_std.pkl', 'rb') as file:
    grid = pickle.load(file)

In [4]:
grid_avg.memory_usage(deep=True).sum(), grid.memory_usage(deep=True).sum()

(461223789, 455096105)

In [34]:
grid_large_std_v2 = grid_avg.copy(deep=True)

In [35]:
grid_large_std_v2 = grid_large_std_v2.reset_index(drop=True)

In [36]:
for col in grid_large_std_v2.select_dtypes(include=['int']).columns:
    grid_large_std_v2[col] = pd.to_numeric(grid_large_std_v2[col], downcast='integer')
    
# For float columns
for col in grid_large_std_v2.select_dtypes(include=['float']).columns:
    grid_large_std_v2[col] = pd.to_numeric(grid_large_std_v2[col], downcast='float')

In [37]:
grid_large_std_v2.memory_usage(deep=True).sum()

180532970

In [38]:
grid_large_std_v2.to_parquet('grid_large_std_v2.parquet')

In [2]:
v2 = pd.read_parquet('grid_large_std_v2.parquet')

In [3]:
v2

,color,gradient,p_slope,ones,longitude,latitude,geometry,cdl_val_2010,cdl_val_2015,cdl_val_2020,...,46000,46003,46006,46007,46100,48400,53700,56800,gradient_flag,p_slope_flag
0,#8c6a18,0.0,0.0,1,-122.559174,40.134506,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,1.128056,0.868350,0.804702,...,0,0,0,0,0,0,0,0,1,1
1,#1a8ed1,0.0,0.0,1,-122.559174,40.141373,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,0.782103,0.868350,0.804702,...,0,0,0,0,0,0,0,0,1,1
2,#cb43be,0.0,0.0,1,-122.559174,40.148243,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,1.128056,0.868350,0.804702,...,0,0,0,0,0,0,0,0,1,1
3,#1eded1,0.0,0.0,1,-122.559174,40.161972,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,1.128056,0.868350,0.804702,...,0,0,0,0,0,0,0,0,1,1
4,#9fdb09,0.0,0.0,1,-122.559174,40.168839,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,1.128056,0.868350,0.804702,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26408,#23a270,0.0,0.0,1,-121.606964,38.501446,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...",-0.529636,-0.579176,-1.476969,...,0,0,0,0,0,0,0,0,1,1
26409,#23fe25,0.0,0.0,1,-121.606964,38.536591,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,-1.394518,-0.849593,-0.849126,...,0,0,0,0,0,0,0,0,1,1
26410,#1279cc,0.0,0.0,1,-121.606964,38.543617,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...",-1.365689,-1.501775,-1.476969,...,0,0,0,0,0,0,0,0,1,1
26411,#c8306b,0.0,0.0,1,-121.400352,38.254971,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,-1.365689,-0.579176,-0.971632,...,0,0,1,0,0,0,0,0,1,1


In [5]:
v2[["ones", "longitude", "latitude", "PercentCoarse"]].to_parquet('ones.parquet')

### Doing deeper slices

In [2]:
with open('aem_15_150_grid.pkl', 'rb') as f:
    depth_slices = pickle.load(f)

In [4]:
cvhm = pd.read_csv("GA-sklearn/cvhm_PC.csv")

In [7]:
cvhm_gdf = gpd.GeoDataFrame(cvhm, geometry=gpd.points_from_xy(cvhm.X_COORD, cvhm.Y_COORD))

crs_albers = {
    'proj': 'aea',           # Albers Equal Area Conic projection
    'lat_1': 29.5,           # First standard parallel
    'lat_2': 45.5,           # Second standard parallel
    'lat_0': 23.0,           # Latitude of origin
    'lon_0': -120.0,         # Central meridian
    'x_0': 0,                # False easting
    'y_0': 0,                # False northing
    'datum': 'NAD83',        # Datum
    'units': 'm'             # Linear unit in meters
}

cvhm_gdf = cvhm_gdf.set_crs(crs_albers)

In [11]:
cvhm_cols = ["PC_D75", "PC_D125", "PC_D175", "PC_D225",
             "PC_D275", "PC_D325", "PC_D375", "PC_D425", "PC_D475", "geometry"]

In [15]:
cvhm_gdf.to_crs(depth_slices.crs, inplace=True)

In [105]:
with open('grid_large_std_v2.pkl', 'rb') as f:
    grid_v2 = pickle.load(f)

In [106]:
cvhm_gdf.to_crs(grid_v2.crs, inplace=True)
depth_slices.to_crs(grid_v2.crs, inplace=True)

In [25]:
cvhm_gridded = gpd.sjoin_nearest(grid_v2[["geometry"]], 
                           cvhm_gdf[cvhm_cols], 
                           how='left', 
                           distance_col='distance')

In [109]:
grid_v2 = grid_v2.drop(columns=["avg_PC_D25_z", "index_right"])

In [ ]:
cvhm_gridded = cvhm_gridded.drop(columns="distance")

In [67]:
a = gpd.sjoin_nearest(grid_v2, 
                           cvhm_gridded[["PC_D75", "geometry"]], 
                           how='left', 
                           distance_col='distance')
a = a.drop(columns=["distance", "index_right"])
b = gpd.sjoin_nearest(a, 
                           depth_slices[["PercentCoarse_15_30", "geometry"]],
                           how='left', 
                           distance_col='distance')
b = b.drop(columns=["index_right", "distance"])
b["PC_D75"] = (b["PC_D75"] - b["PC_D75"].mean()) / b["PC_D75"].std()

In [72]:
gpd.sjoin_nearest(grid_v2, 
                           cvhm_gridded[["PC_D75", "geometry"]], 
                           how='left', 
                           distance_col='distance')

TypeError: sjoin_nearest() got an unexpected keyword argument 'max_matches'

In [48]:
aem_depths = ['PercentCoarse_15_30', 'PercentCoarse_30_45',
       'PercentCoarse_45_60', 'PercentCoarse_60_75', 'PercentCoarse_75_90',
       'PercentCoarse_90_105', 'PercentCoarse_105_120',
       'PercentCoarse_120_135', 'PercentCoarse_135_150']
cvhm_depths = ['PC_D75', 'PC_D125', 'PC_D175', 'PC_D225', 'PC_D275', 'PC_D325',
       'PC_D375', 'PC_D425', 'PC_D475']

In [120]:
for aem, krige in tqdm(zip(aem_depths, cvhm_depths), total=9):
    a = gpd.sjoin_nearest(depth_slices[["geometry", aem]], 
                           cvhm_gdf[[krige, "geometry"]],
                           how='left',
                           distance_col='distance')
    a = a.drop(columns=["index_right", "distance"])
    b = pd.merge(grid_v2, a, on="geometry", how="left")
    b[krige] = (b[krige] - b[krige].mean()) / b[krige].std()
    b.to_parquet("deeper/" + aem + ".parquet")

100%|██████████| 9/9 [00:40<00:00,  4.54s/it]


In [3]:
import pandas as pd
a = pd.read_parquet("deeper/PercentCoarse_15_30.parquet")

In [13]:
a.columns[:1200]

Index(['color', 'gradient', 'p_slope', 'ones', 'longitude', 'latitude',
       'geometry', 'cdl_val_2010', 'cdl_val_2015', 'cdl_val_2020',
       ...
       '2015_sep_oct_diff_Qg_idw', '2016_sep_oct_diff_Qg_idw',
       '2017_sep_oct_diff_Qg_idw', '2018_sep_oct_diff_Qg_idw',
       '2019_sep_oct_diff_Qg_idw', '2020_sep_oct_diff_Qg_idw',
       '2021_sep_oct_diff_Qg_idw', '2022_sep_oct_diff_Qg_idw',
       '2023_sep_oct_diff_Qg_idw', '2024_sep_oct_diff_Qg_idw'],
      dtype='object', length=1200)